## Instaling the required packages

In [1]:
import os
import time
import json
import requests
import tiktoken
import pandas as pd
from amz_scraper import AmazonScraper
from bs4 import BeautifulSoup
from selenium import webdriver
# from langchain.prompts import PromptTemplate
# from dotenv import load_dotenv, dotenv_values
# from langchain.docstore.document import Document
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.pydantic_v1 import BaseModel, Field
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_text_splitters import CharacterTextSplitter
# from langchain.chains import LLMChain, StuffDocumentsChain
# from langchain_community.document_loaders import TextLoader
# from langchain.chains.summarize import load_summarize_chain
# from langchain_community.callbacks import get_openai_callback
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain

# load_dotenv()

True

#### Decide about loading new data

In [2]:
# importing inputs from the UI 
from user_inputs import new_data_st, max_cus, product
# new_data_st = "Yes"
# max_cus = 15
# product = "ipad air"

2024-04-30 22:22:26.392 
  command:

    streamlit run c:\Users\Mast_Nijanand\anaconda3\envs\aiapp\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:
print(product)
print(max_cus)
print(new_data_st)

''

#### Setting up small vs. large content code toggle

In [3]:
if new_data_st == "Yes":
    new_data = True
else:
    new_data = False

In [4]:
# Setting up logical code block execution toggle
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

#### Getting API Keys

In [5]:
# Activating the API Keys
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
X_RapidAPI_Key = os.environ.get("X_RapidAPI_Key")
Rapid_AI_URL = os.environ.get("Rapid_AI_URL")
Rapid_AI_Host = os.environ.get("Rapid_AI_Host")

## New Product Selection

### Getting Amazon Reivews

In [6]:
if new_data:
    search_query = str(product)   # 'premier protein shake, chocolate'
    scraper = AmazonScraper()
    reviews = scraper.get_closest_product_reviews(search_query, num_reviews = max_cus, debug=True)
    print(reviews)

https://www.amazon.ca/s?k=ipad+air


Exception ignored in: <function Service.__del__ at 0x000001A2D6082560>
Traceback (most recent call last):
  File "c:\Users\Mast_Nijanand\anaconda3\envs\aiapp\lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\Users\Mast_Nijanand\anaconda3\envs\aiapp\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\Mast_Nijanand\anaconda3\envs\aiapp\lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\Mast_Nijanand\anaconda3\envs\aiapp\lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
  File "c:\Users\Mast_Nijanand\anaconda3\envs\aiapp\lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
  File "c:\Users\Mast_Nijanand\anaconda3\envs\aiapp\lib\urllib\request.py", line 536, in _open
    result =

TypeError: object of type 'NoneType' has no len()

In [ ]:
search_query

NameError: name 'search_query' is not defined

In [ ]:
print("Total number of reviewes received: ", len(reviews))

NameError: name 'reviews' is not defined

In [ ]:
import sys
sys.exit(0)

In [ ]:
# Transfering the webscrapped data into a dataframe
if new_data:
    df = pd.DataFrame.from_dict(reviews)
    df.sort_values(by=["rating"], ascending=False, inplace=True)
    df.reset_index(inplace=True)
    df.to_pickle("amazon_reviews_df")  # storing df to ease its future usage
    df.head(5)
else:
    df = pd.read_pickle("amazon_reviews_df")

In [ ]:
df

,index,customer_name,rating,review
0,96,Patricia,5,I was skeptical about buying on Amazon. But th...
1,81,Orakey,5,A little bit of discount for a year's old mode...
2,67,Tumbilah,5,Camera quality is amazingCharges wellGood stor...
3,68,Kayannaphora,5,I really wanted the S23 Ultra because I like t...
4,69,Regina Daniels,5,I wrote a review earlier BUT I did not know a ...
...,...,...,...,...
92,26,Karina M,1,False Advertisement
93,27,Clair Wiederholt,1,Amazon is deducting a $132.60 re-stocking fee ...
94,28,Sharon,1,"My phone is defective. Screen flickers, slow s..."
95,29,James Moses,1,The ad says unlocked for all carriers. This ph...


### Preparing Review Data for Display 

# extracting API reponse in json file
temp_dict = response.json()

# declaring dataframe to collect final review data
df = pd.DataFrame()

df["review"] = [t["body"] for t in temp_dict["reviews"]]
df["review date"] = [(t["date"]) for t in temp_dict["reviews"]]
df["ratings"] = [t["rating"] for t in temp_dict["reviews"]]
df["review title"] = [t["title"] for t in temp_dict["reviews"]]
df["user name"]= [t["user-name"] for t in temp_dict["reviews"]]
df["verified"] = [t["verified"] for t in temp_dict["reviews"]]

# triming trailiing text from date column 
df["review date"] = df["review date"].str[32:]

# diplaying dataframe data
df

## Review Summary Generation

#### Develop Data Summary

In [ ]:
# Developing Summary of Reviews for Each Web
amz_cust_reviews = df["review"]
amz_reviews_str = "".join(each for  each in amz_cust_reviews)
print(f"Total length of all reviews text chain is {len(amz_reviews_str)} characters.")

26721


In [ ]:
# Storing review data into different formats 
# converting the dataframe to CSV format for checking purpose
if new_data:
    df.to_csv("amz_reviews.csv", mode="w", index=False)                       # storing in CSV format
    file = open('./review_docs/amz_reviews.txt','w', encoding='utf-8')        # storing in text format
    file.writelines(amz_reviews_str)
    file.close()

In [ ]:
# Setting LLM 
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
print("LLM gets loaded successfully")

#### Check for Review Content Length

In [ ]:
# Counting AutoScraper output tokens

def count_tokens(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""

    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

total_tokens = count_tokens(str(amz_reviews_str), "cl100k_base")
print("Total actual number of input tokens =", total_tokens)

Total actual number of input tokens = 6026


In [ ]:
if total_tokens <= 3500: 
    print("Reveiew summary will be generated using the small content context method.")
    
    summary_statement = """You are an expeienced copy writer providing a world-class summary of product reviews {reviews} from numerous customers \
                        on a given product from different leading e-commerce platforms. You write summary of all reviews for a target audience \
                        of wide array of product reviewers ranging from a common man to an expeirenced product review professional."""
    summary_prompt = PromptTemplate(input_variables = ["reviews"], template=summary_statement)
    llm_chain = LLMChain(llm=llm, prompt=summary_prompt)
    amz_review_summary_smp = llm_chain.run(amz_reviews_str)
    print("Amazon Review Summary: \n\n", amz_review_summary_smp)

##### Define Function for Sentiment Analysis

In [ ]:
# define a function for sentiment analysis
# https://python.langchain.com/docs/use_cases/tagging/

class Classification(BaseModel):
    Overall_Sentiment: str = Field(..., enum=["Positive", "Neutral", "Negative"])
    Review_Aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    
tagging_prompt = ChatPromptTemplate.from_template(
    """
    Extract the  properties mentioned in the 'Classification' function from the following text.
    Paragraph:
    {input}
    """
)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

### Generating Customer Review Sentiment for smaller inputs

In [ ]:
if total_tokens <= 3500:
    output_smp = tagging_chain.invoke({"input": amz_review_summary_smp})
    print("Customer Reviews' Sentiment \n\n", output_smp)
    print("\n *** PROGRAM EXECUTION ABORTED HERE ***")
    raise StopExecution

In [ ]:
# Splitting the doc into sizeable chunks

raw_documents = TextLoader("./review_docs/amz_reviews.txt", encoding='utf-8').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=0, separators=["\n\n", "\n", " ", ""])
split_text = text_splitter.split_documents(raw_documents)
# docs = [Document(page_content=each) for each in split_text]
print("Total number of documents =", len(docs))

Total number of documents = 24


In [ ]:
print(docs[:2])

[Document(page_content='I was skeptical about buying on Amazon. But this is a legit Samsung s23 ultra. Works with the mobile in the United States. At least in Missouri.A little bit of discount for a year\'s old model but major functions are pretty much the same.Camera quality is amazingCharges wellGood storageDual SIMValue for moneyI really wanted the S23 Ultra because I like the curved edges, which my old phone also had, but no phone carrier or store had them in stock. I was nervous about ordering a phone off Amazon, but my order came fast, in the factory packaging, with no issues and a nice little gift (a charger) from the seller. Plus, the phone is everything I wanted: battery lasts for days, tons of storage, simple to transfer data from my old Galaxy, and charges quickly. All I had to do to set it up was transfer the SIM card. Overall, great buy.I wrote a review earlier BUT I did not know a charger did not come with the cell phone.  Normally a SIM Card is included but it was not.  

### Apply Map Reduce Method 
#### (Summarize large Document)

In [ ]:
# Applying map reduce to summarize large document
# https://python.langchain.com/docs/use_cases/summarization/
print("Map Reduce Process is initiated now") 

map_template = """Based on the following docs {docs}, please provide summary of reviews presented in these documents. 
Review Summary is:"""

map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

The ReduceDocumentsChain handles taking the document mapping results and reducing them into a single output. It wraps a generic CombineDocumentsChain (like StuffDocumentsChain) but adds the ability to collapse documents before passing it to the CombineDocumentsChain if their cumulative size exceeds token_max. In this example, we can actually re-use our chain for combining our docs to also collapse our docs.

So if the cumulative number of tokens in our mapped documents exceeds 4000 tokens, then we’ll recursively pass in the documents in batches of \< 4000 tokens to our StuffDocumentsChain to create batched summaries. And once those batched summaries are cumulatively less than 4000 tokens, we’ll pass them all one last time to the StuffDocumentsChain to create the final summary.

In [ ]:
# Reduce
reduce_template = """The following is set of summaries: 
{doc_summaries}
Take these document and return your consolidated summary in a professional manner addressing the key points of the customer reviews. 
Review Summary is:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="doc_summaries")

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=3500,
)

Combining our map and reduce chains into one

In [ ]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

#### Generating Map Reduce Summary

In [ ]:
amz_review_summary_mr = map_reduce_chain.invoke(docs)

In [ ]:
print("Amazon Review Summary as per Map Reduced Method: \n\n", amz_review_summary_mr['input_documents'][0])

page_content='I was skeptical about buying on Amazon. But this is a legit Samsung s23 ultra. Works with the mobile in the United States. At least in Missouri.A little bit of discount for a year\'s old model but major functions are pretty much the same.Camera quality is amazingCharges wellGood storageDual SIMValue for moneyI really wanted the S23 Ultra because I like the curved edges, which my old phone also had, but no phone carrier or store had them in stock. I was nervous about ordering a phone off Amazon, but my order came fast, in the factory packaging, with no issues and a nice little gift (a charger) from the seller. Plus, the phone is everything I wanted: battery lasts for days, tons of storage, simple to transfer data from my old Galaxy, and charges quickly. All I had to do to set it up was transfer the SIM card. Overall, great buy.I wrote a review earlier BUT I did not know a charger did not come with the cell phone.  Normally a SIM Card is included but it was not.  The small 

### Apply Refine Method 
#### (Summarize large Document)

In [ ]:
# Checking the Refine Method for comparison
# https://medium.com/@abonia/summarization-with-langchain-b3d83c030889
print("Document Refine Method is initiated now")

prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in that covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"])

# Load refine chain
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_text",
   output_key="output_text",
)
amz_review_summary_ref = chain.invoke({"input_text": docs}, return_only_outputs=True)

#### Generating Refine Method Summary

In [ ]:
print("Amazon Review Summary as per Refine Method: \n\n", amz_review_summary_ref['intermediate_steps'][0])

The text discusses the purchase of a Samsung S23 Ultra from Amazon. The buyer was initially skeptical but found the phone to be legitimate and compatible with mobile carriers in the United States. The phone had good camera quality, charging capabilities, storage, and dual SIM functionality. The buyer was happy with the purchase as it met their expectations and came with a charger as a gift from the seller. However, there were some issues with the delivery and missing items such as a charger and SIM card.


### Customer Sentiment from Review Summaries

In [ ]:
# Generating customer reviews sentiment based on map reduce method summary
output_mr = tagging_chain.invoke({"input": amz_review_summary_mr['input_documents'][0]})

In [ ]:
print("Sentiment Output for Map Reduce Summary :", output_mr)

Sentiment Output for Map Reduce Summary : Overall_Sentiment='Positive' Review_Aggressiveness=3


In [ ]:
# Generating customer reviews sentiment based on refine method summary
output_ref = tagging_chain.invoke({"input": amz_review_summary_ref})

In [ ]:
print("Sentiment Output for Refined Method Summary :", output_ref)

Sentiment Output for Refined Method Summary : Overall_Sentiment='Neutral' Review_Aggressiveness=3


### Generating Comparative Output Summary

In [ ]:
print("Generating Review Summary and Sentiment Output Dataframe now")

data_output = pd.DataFrame({"Review Summary":[amz_review_summary_mr["input_documents"][0], amz_review_summary_ref["intermediate_steps"][0]], 
                            "Sentiments":[output_mr, output_ref]},
                           index=(["Map Reduce Method", "Refine Method"]))

pd.set_option("display.colheader_justify","center")
pd.set_option('display.max_colwidth', None)

### Exporting Summary

In [ ]:
data_output.to_pickle("data_output")
data_output.to_csv("data output.csv", mode="w", index=False)

### Displaying Summary

In [ ]:
data_output = data_output.style.set_properties(**{'text-align': 'left'})
data_output.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

print("Final Data Output generation has been complete now")
data_output

,Review Summary,Sentiments
Map Reduce Method,"page_content='I was skeptical about buying on Amazon. But this is a legit Samsung s23 ultra. Works with the mobile in the United States. At least in Missouri.A little bit of discount for a year\'s old model but major functions are pretty much the same.Camera quality is amazingCharges wellGood storageDual SIMValue for moneyI really wanted the S23 Ultra because I like the curved edges, which my old phone also had, but no phone carrier or store had them in stock. I was nervous about ordering a phone off Amazon, but my order came fast, in the factory packaging, with no issues and a nice little gift (a charger) from the seller. Plus, the phone is everything I wanted: battery lasts for days, tons of storage, simple to transfer data from my old Galaxy, and charges quickly. All I had to do to set it up was transfer the SIM card. Overall, great buy.I wrote a review earlier BUT I did not know a charger did not come with the cell phone. Normally a SIM Card is included but it was not. The small items that should be include are not. My delivery read left in mailbox; ""NO"" the box (Too large for the mailbox) and my mail was left on the front porch. Also, the charge is not included. Basically, you\'re just'",Overall_Sentiment='Positive' Review_Aggressiveness=3
Refine Method,"The text discusses the purchase of a Samsung S23 Ultra from Amazon. The buyer was initially skeptical but found the phone to be legitimate and compatible with mobile carriers in the United States. The phone had good camera quality, charging capabilities, storage, and dual SIM functionality. The buyer was happy with the purchase as it met their expectations and came with a charger as a gift from the seller. However, there were some issues with the delivery and missing items such as a charger and SIM card.",Overall_Sentiment='Neutral' Review_Aggressiveness=3
